In [2]:
import handcalcs.render
import forallpeople
from math import *
forallpeople.environment('structural', top_level = True)
import pyodbc
import pandas as pd

In [3]:
server = 'steel.database.windows.net'
database = 'steeldata'
username = 'adrianrymarz'
password = 'rq7U9zU6P@TtRZRiwurXA'   
driver= '{ODBC Driver 17 for SQL Server}'
conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"
table_name = 'IPE'

conn = pyodbc.connect(conn_str)
if conn:
    print("Połączenie z bazą danych udane.")
cursor = conn.cursor()

Połączenie z bazą danych udane.


In [4]:
column_profile = 'IPE100'
query = f"SELECT h FROM [{table_name}] WHERE name = '{column_profile}'"
cursor.execute(query)
h_s = cursor.fetchone()
if row is not None:
    h_s = row[0]
    print(h_s)
else:
    print("Nie znaleziono pasującego wiersza.")

NameError: name 'row' is not defined

# Dane obliczeniowe

In [35]:
%%render short
#------------Dane Do Obliczeń------------#

#Słup HEA200, S235
b_s = 200*mm
t_w_s = 6.5*mm
t_f_s = 10*mm
r_s = 18*mm
A_s = 5283*mm**2
I_y_s = 36920000*mm**4
f_y_s = 235*N/mm**2
f_u_s = 360*N/mm**2

#Belka IPE300, S235
h_b = 300*mm
b_b = 150*mm
t_w_b = 7.1*mm
t_f_b = 10.7*mm
r_b = 15*mm
A_b = 5381*mm**2
I_y_b = 83560000*mm**4
f_y_b = 235*N/mm**2
f_u_b = 360*N/mm**2

#Blacha 230*110*10, S235
g_v = 35*mm
g_h = 10*mm
h_p = 230*mm
b_p = 110*mm
t_p = 10*mm
f_y_p = 235*N/mm**2
f_u_p = 360*N/mm**2

#Kierunek równoległy do kierunku działania obciążenia (1)
n_1 = 3
e_1 = 45*mm
e_1_b = 80*mm
p_1 = 70*mm

#Kierunek prostopadły do kierunku działania obciążenia (2)
n_2 = 1
e_2 = 50*mm
e_2_b = 50*mm
z = 60*mm

#Śruby niesprężone M20 klasy 8.8
n = n_1 * n_2
klasa = 8.8
A_s = 245*mm**2
d = 20*mm
d_0 = 22*mm
f_yb = 640*N/mm**2
f_ub = 800*N/mm**2

#Spoiny pachwinowe
a = 5*mm

#Częściowe współczynniki bezpieczeństwa
gamma_MO = 1
gamma_M2 = 1.25
gamma_Mu = 1.1

#Obliczeniowa siła ścinająca
V_Ed = 100*kN


#------------Nośność połączenia na ściananie------------#

#Ścięcie trzpienia śrub
if(klasa == 8.8):
    alpha_v = 0.6
if((n_2 == 1) and (n == n_1)):
    alpha = 0
    beta = (6 * z) / (n * (n + 1) * p_1)
F_vRd = (alpha_v * f_ub * A_s) / gamma_M2
V_Rd1 = (n * F_vRd) / (sqrt((1 + alpha * n)**2 + (beta * n)**2))

#Uplastycznienie ścianki otworu na śrubę w blasze węzłowej

alpha_b = min(e_1 / (3 * d_0), p_1 / (3 * d_0) - 0.25, f_ub / f_u_p, 1)
k_1 = min(2.8 * e_2 / d_0 - 1.7, 2.5)
F_bRd_ver = (k_1 * alpha_b * f_u_p * d * t_p) / gamma_M2
alpha_b = min(e_2 / (3 * d_0), f_ub / f_u_p, 1)
k_1 = min(2.8 * e_1 / d_0 - 1.7, 1.4 * p_1 / d_0 - 1.7, 2.5)
F_bRd_hor = (k_1 * alpha_b * f_u_p * d * t_p) / gamma_M2
V_Rd2 = n / (sqrt(((1 + alpha * n) / (F_bRd_ver/kN))**2 + ((beta * n) / (F_bRd_hor/kN))**2))*kN

#Uplastycznienie ścianki otworu na śrubę w środniku belki 

alpha_b = min(p_1 / (3 * d_0) - 0.25, f_ub / f_u_b, 1)
k_1 = min(2.8 * e_2_b / d_0 - 1.7, 2.5)
F_bRd_ver = (k_1 * alpha_b * f_u_b * d * t_w_b) / gamma_M2
alpha_b = min(e_2_b / (3 * d_0), f_ub / f_u_b, 1)
k_1 = min(1.4 * p_1 / d_0 - 1.7, 2.5)
F_bRd_hor = (k_1 * alpha_b * f_u_p * d * t_w_b) / gamma_M2
V_Rd8 = n / (sqrt(((1 + alpha * n) / (F_bRd_ver/kN))**2 + ((beta * n) / (F_bRd_hor/kN))**2))*kN

#------------Nośność połączenia ze względu na ciągnienie------------#

#Ścięcie trzpieni śrub
F_vRdu = alpha_v * f_ub * A_s / gamma_Mu
N_Rdu1 = n * F_vRdu

#Uplastycznienie ścianki otworu na śrubę w blasze węzłowej
alpha_b = min(e_2 / (3 * d_0), f_ub / f_u_p, 1)
k_1 = min(2.8 * e_1 / d_0 - 1.7, 1.4 * p_1 / d_0 - 1.7 , 2.5)
F_bRdu_hor = k_1 * alpha_b * f_u_p * d * t_p / gamma_Mu
N_Rdu2 = n * F_bRdu_hor

#Uplastycznienie ścianki otworu na śrubę w środniku belki
alpha_b = min(e_2_b / (3 * d_0), f_ub / f_u_b, 1)
k_1 = min(1.4 * p_1 / d_0 - 1.7, 2.5)
F_bRdu_hor = k_1 * alpha_b * f_u_b * d * t_w_b / gamma_Mu
N_Rdu5 = n * F_bRdu_hor
conn.close()


TypeError: 'NoneType' object is not subscriptable

In [31]:
%%render short 2
V_Rd1
V_Rd2
V_Rd8
N_Rdu1
N_Rdu2
N_Rdu5

<IPython.core.display.Latex object>